# Chapter 11
http://allendowney.github.io/ThinkStats2/

## Reading

In [1]:
from os.path import basename, exists


def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + local)


download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/thinkstats2.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/thinkplot.py")

In [2]:
import numpy as np
import pandas as pd

import thinkstats2
import thinkplot

### Multiple regression

In [3]:
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/nsfg.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/first.py")

download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemPreg.dct")
download(
    "https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemPreg.dat.gz"
)

In [4]:
import first

live, firsts, others = first.MakeFrames()

#### Using Statsmodels for single regression

In [5]:
import statsmodels.formula.api as smf

In [6]:
formula = 'totalwgt_lb ~ agepreg'
model = smf.ols(formula, data=live)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_lb   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     43.02
Date:                Tue, 05 Jul 2022   Prob (F-statistic):           5.72e-11
Time:                        09:52:30   Log-Likelihood:                -15897.
No. Observations:                9038   AIC:                         3.180e+04
Df Residuals:                    9036   BIC:                         3.181e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      6.8304      0.068    100.470      0.000       6.697       6.964
agepreg        0.0175      0.003      6.559      0.000       0.012       0.023
==============================================================================
Omnibus:                     1024.052   Durbin-Watson:                   1.618
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3081.833
Skew:                          -0.601   Prob(JB):                         0.00
Kurtosis:                       5.596   Cond. No.                         118.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [7]:
inter = results.params['Intercept']
slope = results.params['agepreg']
inter, slope

(6.830396973311053, 0.0174538514718029)

In [8]:
slope_pvalue = results.pvalues['agepreg']
slope_pvalue

5.7229471073126415e-11

In [9]:
results.rsquared

0.004738115474710369

In [10]:
diff_weight = firsts.totalwgt_lb.mean() - others.totalwgt_lb.mean()
diff_weight

-0.12476118453549034

In [11]:
diff_age = firsts.agepreg.mean() - others.agepreg.mean()
diff_age

-3.586434766150152

In [12]:
slope * diff_age

-0.06259709972169493

So the age of the mother accounts for about half of the observed decrease in birthweight for first babies vs. others.

#### Using Statsmodels for a single regression with a categorical variable

In [13]:
live['isfirst'] = live.birthord == 1
formula = 'totalwgt_lb ~ isfirst'
results = smf.ols(formula, data=live).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_lb   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     17.74
Date:                Tue, 05 Jul 2022   Prob (F-statistic):           2.55e-05
Time:                        09:55:46   Log-Likelihood:                -15909.
No. Observations:                9038   AIC:                         3.182e+04
Df Residuals:                    9036   BIC:                         3.184e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           7.3259      0.021    356.007      0.000       7.286       7.366
isfirst[T.True]    -0.1248      0.030     -4.212      0.000      -0.183      -0.067
==============================================================================
Omnibus:                      988.919   Durbin-Watson:                   1.613
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2897.107
Skew:                          -0.589   Prob(JB):                         0.00
Kurtosis:                       5.511   Cond. No.                         2.58
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### Running a multiple regression

In [14]:
formula = 'totalwgt_lb ~ isfirst + agepreg'
results = smf.ols(formula, data=live).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_lb   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     24.02
Date:                Tue, 05 Jul 2022   Prob (F-statistic):           3.95e-11
Time:                        09:56:23   Log-Likelihood:                -15894.
No. Observations:                9038   AIC:                         3.179e+04
Df Residuals:                    9035   BIC:                         3.182e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           6.9142      0.078     89.073      0.000       6.762       7.066
isfirst[T.True]    -0.0698      0.031     -2.236      0.025      -0.131      -0.009
agepreg             0.0154      0.003      5.499      0.000       0.010       0.021
==============================================================================
Omnibus:                     1019.945   Durbin-Watson:                   1.618
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3063.682
Skew:                          -0.599   Prob(JB):                         0.00
Kurtosis:                       5.588   Cond. No.                         137.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

When we control for mother's age, the apparent difference due to `isfirst` is halved.

In [15]:
live['agepreg2'] = live.agepreg**2
formula = 'totalwgt_lb ~ isfirst + agepreg + agepreg2'
results = smf.ols(formula, data=live).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_lb   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     22.64
Date:                Tue, 05 Jul 2022   Prob (F-statistic):           1.35e-14
Time:                        09:57:34   Log-Likelihood:                -15884.
No. Observations:                9038   AIC:                         3.178e+04
Df Residuals:                    9034   BIC:                         3.181e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           5.6923      0.286     19.937      0.000       5.133       6.252
isfirst[T.True]    -0.0504      0.031     -1.602      0.109      -0.112       0.011
agepreg             0.1124      0.022      5.113      0.000       0.069       0.155
agepreg2           -0.0018      0.000     -4.447      0.000      -0.003      -0.001
==============================================================================
Omnibus:                     1007.149   Durbin-Watson:                   1.616
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3003.343
Skew:                          -0.594   Prob(JB):                         0.00
Kurtosis:                       5.562   Cond. No.                     1.39e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.39e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Using age squared, the apparent effect of `isfirst` gets even smaller and loses statistical significance.

### Data mining

In [16]:
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemResp.dct")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemResp.dat.gz")

In [17]:
import nsfg

live = live[live.prglngth>30]
resp = nsfg.ReadFemResp()
resp.index = resp.caseid
join = live.join(resp, on='caseid', rsuffix='_r')
join.shape

(8884, 3333)

In [18]:
join.head()

,caseid,pregordr,howpreg_n,howpreg_p,moscurrp,nowprgdk,pregend1,pregend2,nbrnaliv,multbrth,...,pubassis_i_r,basewgt_r,adj_mod_basewgt_r,finalwgt_r,secu_r,sest_r,cmintvw_r,cmlstyr,screentime,intvlngth
0,1,1,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,3410.389399,3869.349602,6448.271112,2,9,1231,1219,19:56:43,67.563833
1,1,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,3410.389399,3869.349602,6448.271112,2,9,1231,1219,19:56:43,67.563833
2,2,1,NaN,NaN,NaN,NaN,5.0,NaN,3.0,5.0,...,0,7226.301740,8567.549110,12999.542264,2,12,1231,1219,14:54:03,106.018167
3,2,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,7226.301740,8567.549110,12999.542264,2,12,1231,1219,14:54:03,106.018167
4,2,3,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,7226.301740,8567.549110,12999.542264,2,12,1231,1219,14:54:03,106.018167


Note that we don't clean most of the variables, so we will probably miss some good ones.

In [19]:
import patsy

In [20]:
def GoMining(df):
    """
    Searches for variables that predict birth weight.
    `df`: DataFrame of pregnancy records
    returns: list of (rsquared, variable name) pairs
    """
    
    variables = []
    for name in df.columns:
        try:
            if df[name].var() < 1e-7: # ignore variables with no variance - not reliable
                continue
            
            formula = f'totalwgt_lb ~ agepreg + {name}'
            model = smf.ols(formula, data=df)
            if model.nobs < len(df) / 2: # ignore variables with more than half of observations missing
                continue
                
            results = model.fit()
        except(ValueError, TypeError, patsy.PatsyError) as e:
            continue
            
        variables.append((results.rsquared, name))
        
    return variables

In [22]:
variables = GoMining(join)
len(variables), variables[:10]

(890,
 [(0.005357647323640635, 'caseid'),
  (0.005750013985077129, 'pregordr'),
  (0.006330980237390427, 'pregend1'),
  (0.016017752709788224, 'nbrnaliv'),
  (0.005543156193094756, 'cmprgend'),
  (0.005442800591639707, 'cmprgbeg'),
  (0.005327612601561005, 'gestasun_m'),
  (0.007023552638453112, 'gestasun_w'),
  (0.12340041363361076, 'wksgest'),
  (0.027144274639580024, 'mosgest')])

The following functions report the variables with the highest values of $R^2$.

In [23]:
import re

def ReadVariables():
    """
    Reads Stata dictionary files for NSFG data.
    
    returns: DataFrame that maps variable names to descriptions.
    """
    vars1 = thinkstats2.ReadStataDct('2002FemPreg.dct').variables
    vars2 = thinkstats2.ReadStataDct('2002FemResp.dct').variables
    
    all_vars = vars1.append(vars2)
    all_vars.index = all_vars.name
    
    return all_vars

def MiningReport(variables, n=30):
    """
    Prints variables with the highest R^2.
    
    `variables`: list of (R^2, variable name) pairs
    `n`: number of pairs to print
    """
    
    all_vars = ReadVariables()
    
    variables.sort(reverse=True)
    for r2, name in variables[:n]:
        key = re.sub('_r$', '', name)
        try:
            desc = all_vars.loc[key].desc
            if isinstance(desc, pd.Series):
                desc = desc[0]
            print(name, r2, desc)
        except (KeyError, IndexError):
            print(name, r2)

In [24]:
MiningReport(variables)

totalwgt_lb 1.0
birthwgt_lb 0.9498127305978009 BD-3 BIRTHWEIGHT IN POUNDS - 1ST BABY FROM THIS PREGNANCY
lbw1 0.30082407844707704 LOW BIRTHWEIGHT - BABY 1
prglngth 0.13012519488625063 DURATION OF COMPLETED PREGNANCY IN WEEKS
wksgest 0.12340041363361076 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN WEEKS)
agecon 0.10203149928156086 AGE AT TIME OF CONCEPTION
mosgest 0.027144274639580024 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN MONTHS)
babysex 0.018550925293941978 BD-2 SEX OF 1ST LIVEBORN BABY FROM THIS PREGNANCY
race_r 0.016199503586252995 RACE
race 0.016199503586252995 RACE
nbrnaliv 0.016017752709788224 BC-2 NUMBER OF BABIES BORN ALIVE FROM THIS PREGNANCY
paydu 0.014003795578114597 IB-10 CURRENT LIVING QUARTERS OWNED/RENTED, ETC
rmarout03 0.01343006646571343 INFORMAL MARITAL STATUS WHEN PREGNANCY ENDED - 3RD
birthwgt_oz 0.013102457615706053 BD-3 BIRTHWEIGHT IN OUNCES - 1ST BABY FROM THIS PREGNANCY
anynurse 0.012529022541810764 BH-1 WHETHER R BREASTFED THIS CHILD AT ALL - 1ST FROM T

/var/folders/pv/jsnn4l_x60x5zx013kbk8l280000gn/T/ipykernel_4770/863314687.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_vars = vars1.append(vars2)


Some of these, such as duration of breastfeeding, are not useful for prediction because they are not known ahead of time.

Combine the variables that seem to have the most explanatory power:

In [26]:
formula = ('totalwgt_lb ~ agepreg + C(race) + babysex==1 + nbrnaliv>1 + paydu==1 + totincr')
results = smf.ols(formula, data=join).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_lb   R-squared:                       0.060
Model:                            OLS   Adj. R-squared:                  0.059
Method:                 Least Squares   F-statistic:                     79.98
Date:                Tue, 05 Jul 2022   Prob (F-statistic):          4.86e-113
Time:                        10:11:38   Log-Likelihood:                -14295.
No. Observations:                8781   AIC:                         2.861e+04
Df Residuals:                    8773   BIC:                         2.866e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                6.6303      0.065    102.223      0.000       6.503       6.757
C(race)[T.2]             0.3570      0.032     11.215      0.000       0.295       0.419
C(race)[T.3]             0.2665      0.051      5.175      0.000       0.166       0.367
babysex == 1[T.True]     0.2952      0.026     11.216      0.000       0.244       0.347
nbrnaliv > 1[T.True]    -1.3783      0.108    -12.771      0.000      -1.590      -1.167
paydu == 1[T.True]       0.1196      0.031      3.861      0.000       0.059       0.180
agepreg                  0.0074      0.003      2.921      0.004       0.002       0.012
totincr                  0.0122      0.004      3.110      0.002       0.005       0.020
==============================================================================
Omnibus:                      398.813   Durbin-Watson:                   1.604
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1388.362
Skew:                          -0.037   Prob(JB):                    3.32e-302
Kurtosis:                       4.947   Cond. No.                         221.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Race: 1 = black, 2 = white, 3 = other

Babies of black mothers are lighter than babies of other races by 0.27 to 0.36 lb.

Sex: 1 = male

Male babies are heavier by about 0.3 lb.

Babies who are part of a multiple birth are lighter by about 1.4 lb.

Homeowners: `paydu` 1 = homeowner

People who own their homes have babies that are heavier by about 0.12 lb, even though we control for income.

$R^2 = 0.06$, which is pretty small. RMSE w/o the model is 1.27 lb, and RMSE with the model is 1.23, so we don't decrease RMSE by very much using the model.

### Logistic regression

Example: suppose we are trying to predict `y` using explanatory variables `x1` and `x2`.

In [27]:
y = np.array([0, 1, 0, 1])
x1 = np.array([0, 0, 0, 1])
x2 = np.array([0, 1, 1, 1])

According to the logit model, the log odds for the $i$th element of $y$ is

$ \log o = \beta_0 + \beta_1 x_1 + \beta_2 x_2 $

Start with arbitrary guess for $\beta$:

In [33]:
beta = np.array([-1.5, 2.8, 1.1])

In [34]:
x = np.array([np.ones(len(y)), x1, x2])

log_o = np.dot(beta, x)
log_o

array([-1.5, -0.4, -0.4,  2.4])

In [35]:
# Convert to odds
o = np.exp(log_o)
o

array([ 0.22313016,  0.67032005,  0.67032005, 11.02317638])

In [36]:
# Convert to probabilities
p = o / (o + 1)
p

array([0.18242552, 0.40131234, 0.40131234, 0.9168273 ])

The likelihoods of the actual outcomes are $p$ where $y$ is 1 and $1 - p$ where $y$ is 0.

In [37]:
likes = np.where(y, p, 1-p)
likes

array([0.81757448, 0.40131234, 0.59868766, 0.9168273 ])

The likelihood of $y$ given $\beta$ is the product of the likelihoods:

In [39]:
like = np.prod(likes)
like

0.1800933529673034

In logistic regression, we search for the values in $\beta$ that maximize likelihood.

In [40]:
import first
live, firsts, others = first.MakeFrames()
live = live[live.prglngth>30]
live['boy'] = (live.babysex==1).astype(int)

In [41]:
model = smf.logit('boy ~ agepreg', data=live)
results = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.693015
         Iterations 3


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    boy   No. Observations:                 8884
Model:                          Logit   Df Residuals:                     8882
Method:                           MLE   Df Model:                            1
Date:                Tue, 05 Jul 2022   Pseudo R-squ.:               6.144e-06
Time:                        10:33:22   Log-Likelihood:                -6156.7
converged:                       True   LL-Null:                       -6156.8
Covariance Type:            nonrobust   LLR p-value:                    0.7833
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0058      0.098      0.059      0.953      -0.185       0.197
agepreg        0.0010      0.004      0.275      0.783      -0.006       0.009
==============================================================================
"""

The mother's age seems to have a small effect - the parameter of `agepreg` is positive. However, the p-value is 0.783, so this could just be due to chance.

In [42]:
formula = 'boy ~ agepreg + hpagelb + birthord + C(race)'
model = smf.logit(formula, data=live)
results = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.692944
         Iterations 3


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    boy   No. Observations:                 8782
Model:                          Logit   Df Residuals:                     8776
Method:                           MLE   Df Model:                            5
Date:                Tue, 05 Jul 2022   Pseudo R-squ.:               0.0001440
Time:                        10:35:20   Log-Likelihood:                -6085.4
converged:                       True   LL-Null:                       -6086.3
Covariance Type:            nonrobust   LLR p-value:                    0.8822
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -0.0301      0.104     -0.290      0.772      -0.234       0.173
C(race)[T.2]    -0.0224      0.051     -0.439      0.660      -0.122       0.077
C(race)[T.3]    -0.0005      0.083     -0.005      0.996      -0.163       0.162
agepreg         -0.0027      0.006     -0.484      0.629      -0.014       0.008
hpagelb          0.0047      0.004      1.112      0.266      -0.004       0.013
birthord         0.0050      0.022      0.227      0.821      -0.038       0.048
================================================================================
"""

To make a prediction, we have to extract the exogenous and endogenous variables.

In [43]:
endog = pd.DataFrame(model.endog, columns=[model.endog_names])
exog = pd.DataFrame(model.exog, columns=model.exog_names)

The baseline prediction strategy is to guess "boy".  In that case, we're right almost 51% of the time.

In [44]:
actual = endog['boy']
baseline = actual.mean()
baseline

0.507173764518333

If we use the previous model, we can compute the number of predictions we get right.

In [45]:
predict = (results.predict() >= 0.5)
true_pos = predict * actual
true_neg = (1 - predict) * (1 - actual)
sum(true_pos), sum(true_neg)

(3944.0, 548.0)

Accuracy:

In [47]:
acc = (sum(true_pos) + sum(true_neg)) / len(actual)
acc

0.5115007970849464

This is slightly higher than the baseline.

## Exercises

**Exercise:** Suppose one of your co-workers is expecting a baby and you are participating in an office pool to predict the date of birth. Assuming that bets are placed during the 30th week of pregnancy, what variables could you use to make the best prediction? You should limit yourself to variables that are known before the birth, and likely to be available to the people in the pool.

In [48]:
import first
live, firsts, others = first.MakeFrames()
live = live[live.prglngth>30]

To predict the date of birth, we want to predict the duration of the completed pregnancy in weeks.

In [68]:
all_vars = ReadVariables()

/var/folders/pv/jsnn4l_x60x5zx013kbk8l280000gn/T/ipykernel_4770/863314687.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_vars = vars1.append(vars2)


In [70]:
all_vars.head()

,start,type,name,fstring,desc,end
name,,,,,,
caseid,1,<class 'str'>,caseid,%12s,RESPONDENT ID NUMBER,13.0
pregordr,13,<class 'int'>,pregordr,%2f,PREGNANCY ORDER (NUMBER),15.0
howpreg_n,15,<class 'int'>,howpreg_n,%2f,BB-2 # OF WEEKS OR MONTHS CURRENTLY PREGNANT,17.0
howpreg_p,17,<class 'int'>,howpreg_p,%1f,BB-2 CURRENT PREGNANCY LENGTH REPORTED IN MONT...,18.0
moscurrp,18,<class 'int'>,moscurrp,%1f,NUMBER OF MONTHS CURRENTLY PREGNANT,19.0


In [73]:
def GoMining(df):
    """
    Searches for variables that predict duration of completed pregnancy in weeks.
    `df`: DataFrame of pregnancy records
    returns: list of (rsquared, variable name) pairs
    """
    
    variables = []
    for name in df.columns:
        try:
            try:
                if 'COMPLETED PREGNANCY' in all_vars.loc[name]['desc']:
                    continue
            except:
                pass
            
            if df[name].var() < 1e-7: # ignore variables with no variance - not reliable
                continue
            
            formula = f'prglngth ~ {name}'
            model = smf.ols(formula, data=df)
            if model.nobs < len(df) / 2: # ignore variables with more than half of observations missing
                continue
                
            results = model.fit()
        except(ValueError, TypeError, patsy.PatsyError) as e:
            continue
            
        variables.append((results.rsquared, name))
        
    return variables

In [74]:
variables = GoMining(live)
MiningReport(variables)

totalwgt_lb 0.12445743148120225
birthwgt_lb 0.11977307804917303 BD-3 BIRTHWEIGHT IN POUNDS - 1ST BABY FROM THIS PREGNANCY
lbw1 0.1037254220458329 LOW BIRTHWEIGHT - BABY 1
prglngth_i 0.022053775796467945 PRGLNGTH IMPUTATION FLAG
nbrnaliv 0.004577565785530036 BC-2 NUMBER OF BABIES BORN ALIVE FROM THIS PREGNANCY
anynurse 0.0024520248837122116 BH-1 WHETHER R BREASTFED THIS CHILD AT ALL - 1ST FROM THIS PREG
bfeedwks 0.002369183944665787 DURATION OF BREASTFEEDING IN WEEKS
pregend1 0.002249389433800819 BC-1 HOW PREGNANCY ENDED - 1ST MENTION
cmlastlb 0.0020431424422022726 CM FOR R'S MOST RECENT LIVE BIRTH
fmarcon5_i 0.0019681593242563133 FMARCON5 IMPUTATION FLAG
evuseint 0.0018917527758608443 EG-1 USE ANY METHOD IN PREGNANCY INTERVAL?
gestasun_m 0.0016571319550157115 BC-5 GESTATIONAL LENGTH OF PREGNANCY IN MONTHS
sest 0.0013223681981655577 SCRAMBLED VERSION OF THE STRATUM
matchfound 0.001309107377129859 CHECK ON WHETHER CHILD MATCHES BIO CHILD IN HH ROSTER - 1ST
cmlstprg 0.0012828619646411132 

/var/folders/pv/jsnn4l_x60x5zx013kbk8l280000gn/T/ipykernel_4770/863314687.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_vars = vars1.append(vars2)


The variables with the highest explanatory value that are known ahead of time are:
- `nbrnaliv` number of babies born alive (sort of, don't know if they will be born alive)
- `birthord` birth order

Check for race:

In [75]:
formula = ('prglngth ~ C(race)')
results = smf.ols(formula, data=join).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               prglngth   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     5.752
Date:                Tue, 05 Jul 2022   Prob (F-statistic):            0.00319
Time:                        11:14:51   Log-Likelihood:                -18293.
No. Observations:                8884   AIC:                         3.659e+04
Df Residuals:                    8881   BIC:                         3.661e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       38.7798      0.039    986.735      0.000      38.703      38.857
C(race)[T.2]     0.1483      0.047      3.185      0.001       0.057       0.240
C(race)[T.3]     0.0232      0.078      0.297      0.766      -0.130       0.176
==============================================================================
Omnibus:                     1599.505   Durbin-Watson:                   1.631
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6170.529
Skew:                          -0.860   Prob(JB):                         0.00
Kurtosis:                       6.703   Cond. No.                         5.20
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [90]:
# Solution from the book

# The following are the only variables I found that have a statistically significant effect on pregnancy length.

import statsmodels.formula.api as smf
model = smf.ols('prglngth ~ birthord==1 + race==2 + nbrnaliv>1', data=live)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               prglngth   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     34.28
Date:                Tue, 05 Jul 2022   Prob (F-statistic):           5.09e-22
Time:                        11:23:33   Log-Likelihood:                -18247.
No. Observations:                8884   AIC:                         3.650e+04
Df Residuals:                    8880   BIC:                         3.653e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                38.7617      0.039   1006.410      0.000      38.686      38.837
birthord == 1[T.True]     0.1015      0.040      2.528      0.011       0.023       0.180
race == 2[T.True]         0.1390      0.042      3.311      0.001       0.057       0.221
nbrnaliv > 1[T.True]     -1.4944      0.164     -9.086      0.000      -1.817      -1.172
==============================================================================
Omnibus:                     1587.470   Durbin-Watson:                   1.619
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6160.751
Skew:                          -0.852   Prob(JB):                         0.00
Kurtosis:                       6.707   Cond. No.                         10.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**Exercise:** The Trivers-Willard hypothesis suggests that for many mammals the sex ratio depends on “maternal condition”; that is, factors like the mother’s age, size, health, and social status. See https://en.wikipedia.org/wiki/Trivers-Willard_hypothesis

Some studies have shown this effect among humans, but results are mixed. In this chapter we tested some variables related to these factors, but didn’t find any with a statistically significant effect on sex ratio.

As an exercise, use a data mining approach to test the other variables in the pregnancy and respondent files. Can you find any factors with a substantial effect?

In [86]:
# Solution

def GoMining(df):
    """Searches for variables that predict birth weight.

    df: DataFrame of pregnancy records

    returns: list of (rsquared, variable name) pairs
    """
    df['boy'] = (df.babysex==1).astype(int)
    variables = []
    for name in df.columns:
        try:
            if df[name].var() < 1e-7:
                continue

            formula='boy ~ agepreg + ' + name
            model = smf.logit(formula, data=df)
            nobs = len(model.endog)
            if nobs < len(df)/2:
                continue

            results = model.fit()
        except:
            continue

        variables.append((results.prsquared, name))

    return variables

variables = GoMining(join)

Optimization terminated successfully.
         Current function value: 0.692991
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692961
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692849
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692996
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692903
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692724
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.693014
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692992
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693010
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692985
  

/Users/lealia/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Optimization terminated successfully.
         Current function value: 0.692973
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692973
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692810
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693014
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693015
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693011
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693003
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692995
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692979
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693014
  

/Users/lealia/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Optimization terminated successfully.
         Current function value: 0.692774
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692999
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692861
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692705
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692723
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692803
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692956
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692786
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693007
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692989
  

...

         Current function value: 0.692696
         Iterations: 35
Optimization terminated successfully.
         Current function value: 0.692993
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693009
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693011
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692853
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692971
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692639
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692917
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692760
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693013
         Iterations 3
Optimization ter

/Users/lealia/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Optimization terminated successfully.
         Current function value: 0.693014
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692888
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692770
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692992
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692976
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692866
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692992
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692976
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692866
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693010
  

/Users/lealia/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Optimization terminated successfully.
         Current function value: 0.692997
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692795
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692693
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692457
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692815
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693002
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693011
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692989
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693008
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693011
  

In [87]:
MiningReport(variables)

totalwgt_lb 0.009696855253233383
birthwgt_lb 0.009274460080281988 BD-3 BIRTHWEIGHT IN POUNDS - 1ST BABY FROM THIS PREGNANCY
constat3 0.0010985419170438382 3RD PRIORITY CODE FOR CURRENT CONTRACEPTIVE STATUS
lbw1 0.0010519527860076705 LOW BIRTHWEIGHT - BABY 1
nplaced 0.001010368752280555 # OF R'S BIO CHILDREN SHE PLACED FOR ADOPTION (BASED ON BPA)
fmarout5 0.0009096579032891183 FORMAL MARITAL STATUS AT PREGNANCY OUTCOME
rmarout6 0.000818252143711895 INFORMAL MARITAL STATUS AT PREGNANCY OUTCOME - 6 CATEGORIES
infever 0.0008115919859909004 EVER USED INFERTILITY SERVICES OF ANY KIND
frsteatd 0.0007675331422082321 AGE (IN MOS) WHEN 1ST SUPPLEMENTED - 1ST FROM THIS PREG
splstwk1 0.0007334122339932581 IF-1 H/P DOING WHAT LAST WEEK (EMPLOYMENT STATUS) 1ST MENTION
pmarpreg 0.0007245809157658822 WHETHER PREGNANCY ENDED BEFORE R'S 1ST MARRIAGE (PREMARITALLY)
usefstp 0.0007122387685902787 EF-3 USE METHOD AT FIRST SEX WITH 1ST PARTNER IN PAST 12 MONTHS?
outcom02 0.0007015744602576479 OUTCOME OF PREG

/var/folders/pv/jsnn4l_x60x5zx013kbk8l280000gn/T/ipykernel_4770/863314687.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_vars = vars1.append(vars2)


In [89]:
# Solution from book

# Eliminating variables that are not known during pregnancy and 
# others that are fishy for various reasons, here's the best model I could find:

formula = 'boy ~ agepreg + fmarout5==5 + infever==1'
model = smf.logit(formula, data=join)
results = model.fit()
results.summary() 

Optimization terminated successfully.
         Current function value: 0.691874
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    boy   No. Observations:                 8884
Model:                          Logit   Df Residuals:                     8880
Method:                           MLE   Df Model:                            3
Date:                Tue, 05 Jul 2022   Pseudo R-squ.:                0.001653
Time:                        11:23:05   Log-Likelihood:                -6146.6
converged:                       True   LL-Null:                       -6156.8
Covariance Type:            nonrobust   LLR p-value:                 0.0001432
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                -0.1805      0.118     -1.534      0.125      -0.411       0.050
fmarout5 == 5[T.True]     0.1582      0.049      3.217      0.001       0.062       0.255
infever == 1[T.True]      0.2194      0.065      3.374      0.001       0.092       0.347
agepreg                   0.0050      0.004      1.172      0.241      -0.003       0.013
=========================================================================================
"""

**Exercise:** If the quantity you want to predict is a count, you can use Poisson regression, which is implemented in StatsModels with a function called `poisson`. It works the same way as `ols` and `logit`. As an exercise, let’s use it to predict how many children a woman has born; in the NSFG dataset, this variable is called `numbabes`.

Suppose you meet a woman who is 35 years old, black, and a college graduate whose annual household income exceeds $75,000. How many children would you predict she has born?

In [104]:
formula = 'numbabes ~ age_r + C(educat) + C(race) + C(totincr)'
model = smf.poisson(formula, data=join)
results = model.fit()
results.summary() 

Optimization terminated successfully.
         Current function value: 1.680262
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:               numbabes   No. Observations:                 8884
Model:                        Poisson   Df Residuals:                     8857
Method:                           MLE   Df Model:                           26
Date:                Tue, 05 Jul 2022   Pseudo R-squ.:                 0.03499
Time:                        16:23:47   Log-Likelihood:                -14927.
converged:                       True   LL-Null:                       -15469.
Covariance Type:            nonrobust   LLR p-value:                1.152e-211
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.6064      0.047     12.854      0.000       0.514       0.699
C(educat)[T.10]     -0.0787      0.027     -2.871      0.004      -0.132      -0.025
C(educat)[T.11]     -0.0057      0.026     -0.217      0.828      -0.057       0.046
C(educat)[T.12]     -0.1969      0.020    -10.021      0.000      -0.235      -0.158
C(educat)[T.13]     -0.2648      0.025    -10.442      0.000      -0.315      -0.215
C(educat)[T.14]     -0.2988      0.027    -11.216      0.000      -0.351      -0.247
C(educat)[T.15]     -0.2457      0.035     -7.119      0.000      -0.313      -0.178
C(educat)[T.16]     -0.2954      0.029    -10.283      0.000      -0.352      -0.239
C(educat)[T.17]     -0.3808      0.048     -7.966      0.000      -0.474      -0.287
C(educat)[T.18]     -0.3958      0.052     -7.593      0.000      -0.498      -0.294
C(educat)[T.19]     -0.3573      0.055     -6.477      0.000      -0.465      -0.249
C(race)[T.2]        -0.1431      0.015     -9.578      0.000      -0.172      -0.114
C(race)[T.3]        -0.1054      0.025     -4.241      0.000      -0.154      -0.057
C(totincr)[T.2]      0.0089      0.041      0.216      0.829      -0.072       0.090
C(totincr)[T.3]      0.1655      0.042      3.958      0.000       0.084       0.247
C(totincr)[T.4]      0.0721      0.039      1.871      0.061      -0.003       0.148
C(totincr)[T.5]     -0.0105      0.039     -0.269      0.788      -0.087       0.066
C(totincr)[T.6]      0.0786      0.037      2.147      0.032       0.007       0.150
C(totincr)[T.7]     -0.0016      0.038     -0.043      0.965      -0.076       0.072
C(totincr)[T.8]     -0.0846      0.039     -2.165      0.030      -0.161      -0.008
C(totincr)[T.9]     -0.0271      0.039     -0.689      0.491      -0.104       0.050
C(totincr)[T.10]    -0.0634      0.042     -1.516      0.129      -0.145       0.019
C(totincr)[T.11]    -0.1135      0.040     -2.854      0.004      -0.191      -0.036
C(totincr)[T.12]    -0.1342      0.041     -3.295      0.001      -0.214      -0.054
C(totincr)[T.13]    -0.1120      0.042     -2.668      0.008      -0.194      -0.030
C(totincr)[T.14]    -0.1214      0.038     -3.162      0.002      -0.197      -0.046
age_r                0.0206      0.001     20.250      0.000       0.019       0.023
====================================================================================
"""

In [105]:
columns = ['age_r', 'educat', 'race', 'totincr']
new = pd.DataFrame([[35, 16, 1, 14]], columns=columns)
results.predict(new)

0    2.485642
dtype: float64

**Exercise:** If the quantity you want to predict is categorical, you can use multinomial logistic regression, which is implemented in StatsModels with a function called `mnlogit`. As an exercise, let’s use it to guess whether a woman is married, cohabitating, widowed, divorced, separated, or never married; in the NSFG dataset, marital status is encoded in a variable called `rmarital`.

Suppose you meet a woman who is 25 years old, white, and a high school graduate whose annual household income is about $45,000. What is the probability that she is married, cohabitating, etc?

In [106]:
formula = 'rmarital ~ age_r + C(educat) + C(race) + C(totincr)'
model = smf.mnlogit(formula, data=join)
results = model.fit()
results.summary() 

/Users/lealia/opt/anaconda3/lib/python3.9/site-packages/statsmodels/discrete/discrete_model.py:2331: RuntimeWarning: divide by zero encountered in log
  logprob = np.log(self.cdf(np.dot(self.exog,params)))
/Users/lealia/opt/anaconda3/lib/python3.9/site-packages/statsmodels/discrete/discrete_model.py:2332: RuntimeWarning: invalid value encountered in multiply
  return np.sum(d * logprob)
/Users/lealia/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


         Current function value: nan
         Iterations: 35


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:               rmarital   No. Observations:                 8884
Model:                        MNLogit   Df Residuals:                     8749
Method:                           MLE   Df Model:                          130
Date:                Tue, 05 Jul 2022   Pseudo R-squ.:                     nan
Time:                        16:25:04   Log-Likelihood:                    nan
converged:                      False   LL-Null:                       -11579.
Covariance Type:            nonrobust   LLR p-value:                       nan
====================================================================================
      rmarital=2       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            1.6840      0.283      5.954      0.000       1.130       2.238
C(educat)[T.10]      0.6253      0.142      4.414      0.000       0.348       0.903
C(educat)[T.11]      0.0100      0.152      0.066      0.948      -0.289       0.309
C(educat)[T.12]     -0.1666      0.107     -1.556      0.120      -0.376       0.043
C(educat)[T.13]     -0.7014      0.156     -4.507      0.000      -1.006      -0.396
C(educat)[T.14]     -0.4316      0.150     -2.878      0.004      -0.725      -0.138
C(educat)[T.15]     -0.4964      0.201     -2.472      0.013      -0.890      -0.103
C(educat)[T.16]     -1.9622      0.280     -6.997      0.000      -2.512      -1.413
C(educat)[T.17]     -1.2712      0.376     -3.383      0.001      -2.008      -0.535
C(educat)[T.18]     -2.4964      0.721     -3.462      0.001      -3.910      -1.083
C(educat)[T.19]    -30.5285   1.03e+06  -2.97e-05      1.000   -2.01e+06    2.01e+06
C(race)[T.2]        -0.8913      0.090     -9.920      0.000      -1.067      -0.715
C(race)[T.3]        -0.5269      0.137     -3.846      0.000      -0.795      -0.258
C(totincr)[T.2]      0.4221      0.257      1.640      0.101      -0.082       0.926
C(totincr)[T.3]      0.1345      0.272      0.495      0.621      -0.398       0.667
C(totincr)[T.4]      0.1775      0.240      0.740      0.459      -0.292       0.647
C(totincr)[T.5]     -0.2493      0.233     -1.069      0.285      -0.706       0.208
C(totincr)[T.6]     -0.1781      0.223     -0.800      0.423      -0.614       0.258
C(totincr)[T.7]     -0.6076      0.231     -2.626      0.009      -1.061      -0.154
C(totincr)[T.8]     -0.3850      0.227     -1.693      0.090      -0.831       0.061
C(totincr)[T.9]     -0.5202      0.232     -2.239      0.025      -0.976      -0.065
C(totincr)[T.10]    -0.8162      0.254     -3.210      0.001      -1.315      -0.318
C(totincr)[T.11]    -0.7419      0.236     -3.149      0.002      -1.204      -0.280
C(totincr)[T.12]    -1.0816      0.252     -4.287      0.000      -1.576      -0.587
C(totincr)[T.13]    -1.3915      0.285     -4.884      0.000      -1.950      -0.833
C(totincr)[T.14]    -1.1843      0.252     -4.705      0.000      -1.678      -0.691
age_r               -0.0560      0.006     -9.538      0.000      -0.067      -0.044
------------------------------------------------------------------------------------
      rmarital=3       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -7.5740      0.913     -8.298      0.000      -9.363      -5.785
C(educat)[T.10]      0.6412      0.403      1.590      0.112      -0.149       1.432
C(educat)[T.11]      0.9795      0.346      2.830      0.005       0.301       1.658
C(educat)[T.12]      0.3216      0.306      1.051      0.293      -0.278       0.921
C(educat)[T.13]      0.7636      0.342      2.234      0.025       0.094       1.433
C(educat)[T

In [110]:
columns = ['age_r', 'educat', 'race', 'totincr']
new = pd.DataFrame([[25, 12, 2, 11]], columns=columns)
results.predict(
    new
).rename(
    columns={
        0: 'currently married',
        1: 'not married but living with opp sex partner',
        2: 'widowed',
        3: 'divorced',
        4: 'separated for reasons of marital discord',
        5: 'never been married',
    }
)

,currently married,not married but living with opp sex partner,widowed,divorced,separated for reasons of marital discord,never been married
0,0.713108,0.156728,0.000628,0.029947,0.019983,0.079606
